<h1>Gradient Checking</h1>
<ul>
    <li>1-Dimensional Gradient Checking</li>
    <li>N-Dimensional Gradient Checking</li>
</ul>

<h2>Import Dependencies</h2>

In [1]:
import numpy as np
from gradient_checking_utils import *

<h2>1-Dimensional Gradient Checking</h2>

In [2]:
def forward_propagation(x, theta):
    '''
    simulate a simple function runs forward propagation
    
    Args:
    x -- input
    theta -- parameter
    
    Returns:
    J -- output
    '''
    
    J = x * theta
    
    return J

In [3]:
# test the forward propagation

x, theta = 2, 4
J = forward_propagation(x, theta)
print ("J = " + str(J))

J = 8


In [4]:
def backward_propagation(x, theta):
    '''
    simulate a simple function runs backward propagation
    
    Args:
    x -- input
    theta -- parameter
    
    Returns:
    dtheta -- gradient
    '''
    
    dtheta = x
    
    return dtheta

In [5]:
# test the backward propagation

x, theta = 3, 4
dtheta = backward_propagation(x, theta)
print ("dtheta = " + str(dtheta))

dtheta = 3


In [6]:
def gradient_check(x, theta, epsilon, threshold):
    '''
    using gradient check to testify the gradient is correctly calculated
    
    Args:
    theta -- parameter
    J -- forward pass output
    dtheta -- backward prop gradient 
    epsilon -- a very small number
    threshold -- a number to determine whether the gradients difference is small enough
    
    Returns:
    difference -- difference between two gradients
    '''
    
    theta_plus = theta + epsilon
    theta_minus = theta - epsilon
    J_plus = forward_propagation(x, theta_plus)
    J_minus = forward_propagation(x, theta_minus)
    dtheta = backward_propagation(x, theta)
    gradapprox = (J_plus - J_minus) / (2 * epsilon)
    difference = np.linalg.norm((dtheta - gradapprox), ord=None) / (np.linalg.norm(dtheta, ord=None) + np.linalg.norm(gradapprox, ord=None))
    
    if difference <= threshold:
        print('The backward propagation is working fine')
    else:
        print('There is something wrong with the gradient')
    
    return difference

In [7]:
x, theta = 3, 4
difference = gradient_check(x, theta, 1e-7, 2e-7)
print(f'the difference: {difference}')

The backward propagation is working fine
the difference: 7.814075313343006e-11


<h2>N-Dimensional Gradient Checking</h2>

In [8]:
def forward_propagation_n(X, Y, parameters):
    """
    Implements the forward propagation
    
    Arguments:
    X -- training set for m examples
    Y -- labels for m examples 
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3":
    
    Returns:
    cost -- the cost function (logistic cost for m examples)
    cache -- a tuple with the intermediate values (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)

    """
    
    # retrieve parameters
    m = X.shape[1]
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)

    # Cost
    log_probs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1. / m * np.sum(log_probs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

In [9]:
def backward_propagation_n(X, Y, cache):
    """
    Implement the backward propagation
    
    Arguments:
    X -- input datapoint, of shape
    Y -- true "label"
    cache -- cache output from forward_propagation_n()
    
    Returns:
    gradients -- A dictionary with the gradients of the cost with respect to each parameter, activation and pre-activation variables.
    """
    
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1. / m * np.dot(dZ3, A2.T)
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1. / m * np.dot(dZ2, A1.T)
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1. / m * np.dot(dZ1, X.T)
    db1 = 1. / m * np.sum(dZ1, axis=1, keepdims=True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

In [10]:
def gradient_check_n(X, Y, parameters, gradients, epsilon, threshold):
    '''
    check if the backward propagation is correctly implemented
    
    Args:
    X -- training data
    Y -- training label
    parameters -- weights and bais
    gradients -- gradients 
    epsilon -- the increment
    threshold -- difference to determine if the two gradients are close enough
    
    Returns:
    difference -- the difference between two gradients
    '''
    
    parameters_values, _ = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    
    for i in range(num_parameters):
        theta_plus = np.copy(parameters_values)
        theta_plus[i] = theta_plus[i] + epsilon
        J_plus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(theta_plus))
        
        theta_minus = np.copy(parameters_values)
        theta_minus[i] = theta_minus[i] - epsilon
        J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(theta_minus))
        
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)

    difference = np.linalg.norm(grad - gradapprox, ord=None) / (np.linalg.norm(grad, ord=None) + np.linalg.norm(gradapprox, ord=None))
    
    if difference >= threshold:
        print('There is something wrong with the backward propagation')
    else:
        print('Backward propagation works fine')
    
    return difference

In [11]:
np.random.seed(1)
X = np.random.randn(4,3)
Y = np.array([1, 1, 0])
W1 = np.random.randn(5,4) 
b1 = np.random.randn(5,1) 
W2 = np.random.randn(3,5) 
b2 = np.random.randn(3,1) 
W3 = np.random.randn(1,3) 
b3 = np.random.randn(1,1) 
parameters = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3}

In [12]:
cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(X, Y, parameters, gradients, 1e-7, 2e-7)
expected_values = [0.2850931567761623, 1.1890913024229996e-07]
print(f'the difference: {difference}')

Backward propagation works fine
the difference: 1.1885552035482147e-07
